<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

In [1]:
# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

colab = False


TASK = 'SubtaskB'

In [2]:
import os

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    proj_folder = '/content/drive/MyDrive'
else:
    proj_folder = os.getcwd()

%cd {proj_folder}


PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'
if not os.path.isdir(PROJECT_DIR):
  %run 'Make_Datasets.py'

/Users/thomas/afrisenti-10701/baselines
/Users/thomas/afrisenti-10701/baselines
Cloning into 'afrisent-semeval-2023'...
remote: Enumerating objects: 350, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 350 (delta 83), reused 138 (delta 42), pack-reused 143
Receiving objects: 100% (350/350), 15.58 MiB | 3.28 MiB/s, done.
Resolving deltas: 100% (139/139), done.
Data directory found.
Creating directory to store formatted data.
/Users/thomas/afrisenti-10701/baselines/afrisent-semeval-2023/SubtaskA/train
pt Creating directory to store train, dev and test splits.
am Creating directory to store train, dev and test splits.
sw Creating directory to store train, dev and test splits.
yo Creating directory to store train, dev and test splits.
ig Creating directory to store train, dev and test splits.
formatted-train-data skipped!
dz Creating directory to store train, dev and test splits.
README.txt skipped!
ha Creating director

In [3]:
from types import SimpleNamespace
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
import pandas
import pandas as pd
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    PreTrainedTokenizerFast,
    PreTrainedTokenizer,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from tokenizers import SentencePieceBPETokenizer
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from datasets import Features, Value, ClassLabel, load_dataset, Dataset

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

logger = logging.getLogger(__name__)


with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")


np.random.seed(420)
torch.manual_seed(69);

NOTE: Redirects are currently not supported in Windows or MacOs.


In [4]:
folder = ''


if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    proj_folder = '/content/drive/MyDrive'
else:
    proj_folder = os.getcwd()

%cd {proj_folder}


PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'

TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK)
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

TRAINING_DATA_DIR

/Users/thomas/afrisenti-10701/baselines/afrisent-semeval-2023/SubtaskB/formatted-train-data


'/Users/thomas/afrisenti-10701/baselines/afrisent-semeval-2023/SubtaskB/formatted-train-data/afrisent-semeval-2023/SubtaskB'

In [5]:
MAXIMUM_SEQUENCE_LENGTH = 500
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', 'multilingual')
EVAL_DIR = os.path.join(PROJECT_DIR, TASK, 'dev')

In [6]:
# Set seed before initializing model.
set_seed(42069)

# obtain train data
df = pd.read_csv(DATA_DIR + '/train.tsv', sep='\t')
df = df.dropna()
train_dataset = Dataset.from_pandas(df)
label_list = df['label'].unique().tolist()

# obtain dev data
df = pd.read_csv(DATA_DIR + '/dev.tsv', sep='\t')
df = df.dropna()
eval_dataset = Dataset.from_pandas(df)
label_list = df['label'].unique().tolist()

# Labels
num_labels = len(label_list)
print(label_list)

FileNotFoundError: [Errno 2] File /Users/thomas/afrisenti-10701/baselines/afrisent-semeval-2023/SubtaskB/formatted-train-data/afrisent-semeval-2023/SubtaskB/splitted-train-dev-test/multilingual/train.tsv does not exist: '/Users/thomas/afrisenti-10701/baselines/afrisent-semeval-2023/SubtaskB/formatted-train-data/afrisent-semeval-2023/SubtaskB/splitted-train-dev-test/multilingual/train.tsv'

####Tokenization

In [ ]:
tokenizer = SentencePieceBPETokenizer()
tokenizer.train_from_iterator(
    train_dataset['text'] + eval_dataset['text'],
    vocab_size=100000,
    min_frequency=5,
    show_progress=True,
    limit_alphabet=500,
)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer
)
tokenizer.add_special_tokens({'eos_token': '[EOS]'})
tokenizer.pad_token = tokenizer.eos_token

# Preprocessing the datasets
# Padding strategy
padding = "max_length"


label_to_id = None
label_to_id = {v: i for i, v in enumerate(label_list)}

In [ ]:
def preprocess_function(examples):
    texts =(examples['text'],)
    result = tokenizer(*texts, padding=padding, max_length=MAXIMUM_SEQUENCE_LENGTH)
    
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    
    result['length'], result["tokenized"] = [], []
    for input_ids in result['input_ids']:
        toks = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        result['length'].append(len(toks)+2)
        result['tokenized'].append(' '.join(toks))
    return result

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset",
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on validation dataset",
)

In [ ]:
train_dataset, eval_dataset

In [ ]:
train_text, train_labels = train_dataset['tokenized'], train_dataset['label']

In [ ]:
eval_text, eval_labels = eval_dataset['tokenized'], eval_dataset['label']

#### Simple Baselines

Class Proportions (AKA Constant / Majority Classifier Results)

In [ ]:
for l in np.unique(eval_labels):
  print(np.mean(np.array(eval_labels) == l), f1_score([l] * len(eval_labels), eval_labels, average='weighted'))

In [ ]:
# set up unigram, bigram, trigram BOW and TF-IDF

unigram_vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=100000, min_df=7, max_df=0.8)
unigram_vectorizer.fit(train_text)
X_train_unigram = unigram_vectorizer.transform(train_text)
X_test_unigram = unigram_vectorizer.transform(eval_text)

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)
X_test_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_test_unigram)

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100000, min_df=7, max_df=0.8)
bigram_vectorizer.fit(train_text)
X_train_bigram = bigram_vectorizer.transform(train_text)
X_test_bigram = bigram_vectorizer.transform(eval_text)

bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)
X_test_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_test_bigram)

trigram_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=100000, min_df=7, max_df=0.8)
trigram_vectorizer.fit(train_text)
X_train_trigram = trigram_vectorizer.transform(train_text)
X_test_trigram = trigram_vectorizer.transform(eval_text)
trigram_tf_idf_transformer = TfidfTransformer()
trigram_tf_idf_transformer.fit(X_train_trigram)
X_train_trigram_tf_idf =trigram_tf_idf_transformer.transform(X_train_trigram)
X_test_trigram_tf_idf =trigram_tf_idf_transformer.transform(X_test_trigram)



In [ ]:
def AdaDTC(**params):
    base_params = {}
    for k, v in list(params.items()):
        if k[:16] == 'base_estimator__':
            base_params[k[16:]] = v
            del params[k]
    return AdaBoostClassifier(base_estimator=DecisionTreeClassifier(**base_params), **params)

classifiers = [
    (MultinomialNB, [
        {'alpha': 1e-3}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 10}, {'alpha': 1e3},
    ]), 
    (SVC, [
        {'C': 1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 1000},
        {'C': 1000.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 1000}
    ]), 
    (LogisticRegression, [
        {'C': 1, 'max_iter': 1000, 'penalty': 'l2'}
    ]), 
    (RandomForestClassifier, [
        {'criterion': 'gini', 'max_depth': None, 'n_estimators': 500}
    ]), 
    (AdaDTC, [
        {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 10, 
          'base_estimator__splitter': 'best', 'learning_rate': 0.1, 'n_estimators': 200},
        {'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': 3, 
         'base_estimator__splitter': 'random', 'learning_rate': 1, 'n_estimators': 200}
    ]),  
    (MLPClassifier, [
        {'activation': 'relu', 'alpha': 1, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
    ])
    ]

X_train_data = {'unigram counts': X_train_unigram,
                'unigram tf-idf': X_train_unigram_tf_idf,
                'bigram counts': X_train_bigram, 
                'bigram tf-idf': X_train_bigram_tf_idf,
                'trigram counts': X_train_trigram, 
                'trigram tf-idf': X_train_trigram_tf_idf}

X_test_data = {'unigram counts': X_test_unigram,
                'unigram tf-idf': X_test_unigram_tf_idf,
                'bigram counts': X_test_bigram, 
                'bigram tf-idf': X_test_bigram_tf_idf,
                'trigram counts': X_test_trigram, 
                'trigram tf-idf': X_test_trigram_tf_idf}

data = []
def train_and_show_scores(title, model, parameters, k=5):

    X_train, X_test = X_train_data[title], X_test_data[title]
    y_train, y_test = train_labels, eval_labels
    
    
    best_train, best_test, best_f1, best_params = 0, 0, 0, None
    best_preds = []
    for params in parameters:
        train_score, test_score, test_f1 = 0, 0, 0
        for _ in range(k):
            clf = model(**params)
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")
                clf.fit(X_train, y_train)
            train_score += clf.score(X_train, y_train) / k
            test_score += clf.score(X_test, y_test) / k
            preds = clf.predict(X_test)
            test_f1 += f1_score(preds, y_test, average='weighted') / k
        
        if test_f1 > best_f1:
            best_train, best_test, best_f1, best_params = train_score, test_score, test_f1, params
            best_preds = preds
            
    data.append([str(model), title, train_score, test_score, test_f1, str(list(best_preds)), str(list(y_test))])
    
    print(f"""
    {title}
    Train score: {best_train:.3}
    Eval score: {best_test:.3}
    Weighted Test F1: {best_f1:.3}
    Params: {best_params}
    """)

In [ ]:
# train_and_show_scores('trigram tf-idf', MultinomialNB, {'alpha': 1}, 'Naive Bayes')
# train_and_show_scores('trigram tf-idf', SVC,
#                       {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 1000}, 'SVM')
# train_and_show_scores('bigram tf-idf', LogisticRegression,
#                       {'C': 1, 'max_iter': 1000, 'penalty': 'l2'}, 'Logistic Regression')
# train_and_show_scores('bigram counts', RandomForestClassifier,
#                       {'criterion': 'gini', 'max_depth': None, 'n_estimators': 500}, 'Random Forest')
# train_and_show_scores('bigram counts', AdaDTC,
#                       {'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': 3, 
#                        'base_estimator__splitter': 'random', 'learning_rate': 1, 'n_estimators': 200}, 'AdaBoost')
# train_and_show_scores('unigram tf-idf', MLPClassifier,
#                       {'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}, 'MLP')

# df = pd.DataFrame(data, columns=['name', 'feature', 'train_acc', 'test_acc', 'test_f1', 'predictions', 'labels'])
# df.to_csv(f'combined_trad.csv', index=False)

In [ ]:
# Get scores for multiple different models
for model, parameters in classifiers:
    print(model)
    train_and_show_scores('unigram counts', model, parameters)
    train_and_show_scores('unigram tf-idf', model, parameters)
    train_and_show_scores('bigram counts', model, parameters)
    train_and_show_scores('bigram tf-idf', model, parameters)
    train_and_show_scores('trigram counts', model, parameters)
    train_and_show_scores('trigram tf-idf', model, parameters)

df = pd.DataFrame(data, columns=['name', 'feature', 'train_acc', 'test_acc', 'test_f1', 'predictions', 'labels'])
df.to_csv(f'combined_trad.csv', index=False)

In [ ]:
# Clear Memory
del unigram_vectorizer
del X_train_unigram 
del X_test_unigram

del unigram_tf_idf_transformer
del X_train_unigram_tf_idf
del X_test_unigram_tf_idf
                                                             
del bigram_vectorizer
del X_train_bigram
del X_test_bigram

del bigram_tf_idf_transformer
del X_train_bigram_tf_idf
del X_test_bigram_tf_idf

del trigram_vectorizer
del X_train_trigram
del X_test_trigram

del trigram_tf_idf_transformer
del X_train_trigram_tf_idf
del X_test_trigram_tf_idf

del X_train_data
del X_test_data 

del train_text
del eval_text